In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<!-- <table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table> -->

# Orchestrating a workflow to train and deploy a PyTorch text classification model using [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)

## Overview

This notebook is continuation to [Training, Tuning and Deploying a PyTorch Text Classification Model](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/community-content/pytorch_text_classification_using_vertex_sdk_and_gcloud/pytorch-text-classification-vertex-ai-train-tune-deploy.ipynb). In this notebook, we orchestrate a ML workflow on [Vertex Pipelines]([Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) using [Kubeflow Pipelines (KFP) SDK v2](https://www.kubeflow.org/docs/components/pipelines/sdk/v2/). The pipeline runs a [custom training job](https://cloud.google.com/vertex-ai/docs/training/custom-training) on Vertex AI to fine-tune a [Hugging Face Transformers](https://huggingface.co/transformers/) model for sentiment classification task and then deploys the model on [Vertex Prediction](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) using [TorchServe](https://pytorch.org/serve/) to make online prediction requests to the model.

### Dataset

We will be using [IMDB movie review dataset](https://huggingface.co/datasets/imdb) from [Hugging Face Datasets](https://huggingface.co/datasets).

### Objective

How to **orchestrate a ML workflow on Vertex Pipelines using KFP SDK (v2) to train and deploy PyTorch models** and emphasize first class support for training and deploying PyTorch models on Vertex AI. 

### Table of Contents

This notebook covers following sections:

# ##TO-DO##

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* [Notebooks](https://cloud.google.com/notebooks)
* [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)
* [Vertex Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)
* [Vertex Predictions](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)
* [Cloud Storage](https://cloud.google.com/storage)
* [Container Registry](https://cloud.google.com/container-registry)
* [Cloud Build](https://cloud.google.com/build) *[Optional]*

Learn about [Vertex AI Pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage Pricing](https://cloud.google.com/storage/pricing) and [Cloud Build Pricing](https://cloud.google.com/build/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages



On colab, authenticate first:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

Then, install the libraries.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune

We will be using [Vertex SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python) to interact with Vertex AI services. The high-level aiplatform library is designed to simplify common data science workflows by using wrapper classes and opinionated defaults.

#### Install Vertex SDK for Python

In [ ]:
!pip -q install {USER_FLAG} --upgrade kfp
!pip -q install {USER_FLAG} --upgrade google-cloud-pipeline-components 
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [ ]:
!python3 -c "import kfp; print('kfp version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

## Before you begin

This notebook does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). 
Also [enable the Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  rthallam-demo-project


Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
from datetime import datetime

def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")

TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

TIMESTAMP = 20210922235201


### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket as necessary

You will need a Cloud Storage bucket for this example.  If you don't have one that you want to use, you can make one now.


Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where AI Platform (Unified) services are
available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

**Change the bucket name below** before running the next cell.

In [4]:
BUCKET_NAME = "gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants



Define some constants. See the "Before you begin" section of the Managed Pipelines User Guide for information on creating your API key.


In [5]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

USER = 'rthallam' # <---CHANGE THIS
PIPELINE_ROOT = '{}/pipeline_root/{}'.format(BUCKET_NAME, USER)

PIPELINE_ROOT

env: PATH=/home/jupyter/.vscode-server-insiders/bin/f66a3e06bcb9f000e5dc0ad0040ff9b32fc75c78/bin:/usr/local/cuda/bin:/opt/conda/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/pipeline_root/rthallam'

Do some imports:

In [6]:
import json
from typing import NamedTuple, List

from google_cloud_pipeline_components import aiplatform as aip_components
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

from kfp.v2 import compiler
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)
from kfp.v2.google.client import AIPlatformClient

In [7]:
APP_NAME = "finetuned-bert-classifier"

## Define the pipeline components



### 1. Component to create custom training container image

In [ ]:
%%writefile ./custom_container/Dockerfile

# Use pytorch GPU base image
FROM gcr.io/cloud-aiplatform/training/pytorch-gpu.1-7

# set working directory
WORKDIR /app

# Install required packages
RUN pip install google-cloud-storage transformers datasets tqdm cloudml-hypertune

# Copies the trainer code to the docker image.
COPY ./trainer/__init__.py /app/trainer/__init__.py
COPY ./trainer/experiment.py /app/trainer/experiment.py
COPY ./trainer/utils.py /app/trainer/utils.py
COPY ./trainer/metadata.py /app/trainer/metadata.py
COPY ./trainer/model.py /app/trainer/model.py
COPY ./trainer/task.py /app/trainer/task.py

# Set up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

In [ ]:
print(f"{BUCKET_NAME}/{APP_NAME}/train/")

In [ ]:
!gsutil cp ./custom_container/Dockerfile {BUCKET_NAME}/{APP_NAME}/train/

In [ ]:
!gsutil cp -r ./python_package/trainer/ {BUCKET_NAME}/{APP_NAME}/train/
!gsutil ls -Rl {BUCKET_NAME}/{APP_NAME}/train/

In [8]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build"],
    output_component_file="./pipelines/build_custom_train_image.yaml",
)
def build_custom_train_image(
    project: str,
    gs_train_src_path: str,
    model_display_name: str
) -> NamedTuple(
    "Outputs",
    [
        ("training_container_uri", str), 
    ]
):

    from datetime import datetime
    import logging
    import os

    from google.cloud.devtools import cloudbuild_v1 as cloudbuild

    logging.getLogger().setLevel(logging.INFO)
    build_client = cloudbuild.services.cloud_build.CloudBuildClient()

    gs_dockerfile_path = os.path.join(gs_train_src_path, 'Dockerfile')
    gs_train_src_path = os.path.join(gs_train_src_path, 'trainer/')

    build_version = datetime.now().strftime("%Y%m%d%H%M%S")
    image_name = f"pytorch_gpu_train_{model_display_name}"
    training_image_uri = f"gcr.io/{project}/{image_name}:{build_version}"
    logging.info(f"training_image_uri: {training_image_uri}")

    build = cloudbuild.Build(
        images=[training_image_uri]
    )
    build.steps = [
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", "-r", gs_train_src_path, "."]
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", gs_dockerfile_path, "Dockerfile"]
        },
        {
            "name": 'gcr.io/cloud-builders/docker',
            "args": [ 'build', '-t', training_image_uri, '.' ]
        }
    ]
    operation = build_client.create_build(project_id=project, build=build, timeout=1800)
    logging.info("IN PROGRESS:")
    logging.info(operation.metadata)

    result = operation.result()
    # Print the completed status
    logging.info("RESULT:", result.status)
    return(training_image_uri,)

### 2. Component to run training job on Vertex AI

In [9]:
@component(
    # base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest", 
    base_image="python:3.7",
    packages_to_install=["google-cloud-aiplatform", "pandas", "fsspec"],
    output_component_file="./pipelines/submit_custom_training_job.yaml")
def submit_custom_training_job( 
    project: str,
    bucket: str,
    training_container_image_uri: str,
    metrics: Output[Metrics],
    model: Output[Model],
    eval_metric_key: str,
    model_display_name: str
) -> NamedTuple(
    "Outputs",
    [
        ("eval_metric", float),
        ("eval_loss", float)
    ]
):

    from google.cloud import aiplatform
    from datetime import datetime
    import logging

    import pandas as pd

    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

    logging.info(f"Model artifacts located at model.uri = {model.uri}")
    logging.info(f"Model artifacts located at model.path = {model.path}")

    # initialize vertex sdk
    aiplatform.init(
        project=project, 
        staging_bucket=bucket
    )

    JOB_NAME = f"{model_display_name}-pytorch-cstm-cntr-{TIMESTAMP}"

    # configure the job with container image spec
    job = aiplatform.CustomContainerTrainingJob(
        display_name=JOB_NAME, 
        container_uri=training_container_image_uri)

    # define training code arguments
    training_args = [
        "--num-epochs", "2", 
        "--model-name", model_display_name]

    # define worker pool specs
    train_replica_count = 1
    train_machine_type = "n1-standard-8"
    train_accelerator_type = "NVIDIA_TESLA_V100"
    train_accelerator_count = 1

    # output directories
    # base_output_dir = f"{bucket}/{JOB_NAME}"

    # submit the custom job to Vertex training service
    job_response = job.run(
        replica_count=train_replica_count,
        machine_type=train_machine_type,
        accelerator_type=train_accelerator_type,
        accelerator_count=train_accelerator_count,
        base_output_dir=model.uri,
        args=training_args
    )

    start, end = job._gca_resource.start_time, job._gca_resource.end_time

    logging.info(f"Model artifacts located at {model.uri}/model/{model_display_name}")
    model.metadata["framework"] = "pytorch"

    logging.info(f"Model artifacts located at model.uri = {model.uri}")
    logging.info(f"Model artifacts located at model.path = {model.path}")
    model.metadata["time_to_train_in_seconds"] = (end - start).total_seconds()

    # fetch metrics from the training job run
    metrics_uri = f"{model.path}/model/{model_display_name}/all_results.json"
    logging.info(f"Reading and logging metrics from {metrics_uri}")
    metrics_df = pd.read_json(metrics_uri, typ='series')
    for k,v  in metrics_df.items():
        logging.info(f'     {k} -> {v}')
        metrics.log_metric(k, v)

    # capture eval metric
    eval_metric = metrics_df[eval_metric_key] if eval_metric_key in metrics_df.keys() else None
    eval_loss = metrics_df["eval_loss"] if "eval_loss" in metrics_df.keys() else None
    logging.info(f'     {eval_metric_key} -> {eval_metric}')
    logging.info(f'     "eval_loss" -> {eval_loss}')

    return (eval_metric, eval_loss)
    

### 3. Component to create serving container image

In [10]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM pytorch/torchserve:latest-cpu

# install dependencies
RUN pip3 install transformers

# copy model artifacts, custom handler and other dependencies
COPY ./custom_text_handler.py ./index_to_name.json /home/model-server/
COPY ./model/$APP_NAME/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_text_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/tokenizer.json,/home/model-server/training_args.bin,/home/model-server/tokenizer_config.json,/home/model-server/special_tokens_map.json,/home/model-server/vocab.txt,/home/model-server/index_to_name.json" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]
EOF

echo "Writing ./predictor/Dockerfile"

Writing ./predictor/Dockerfile


In [ ]:
!gsutil cp ./predictor/Dockerfile {BUCKET_NAME}/{APP_NAME}/serve/
!gsutil cp ./predictor/custom_text_handler.py ./predictor/index_to_name.json {BUCKET_NAME}/{APP_NAME}/serve/predictor/

In [ ]:
!gsutil ls -lR {BUCKET_NAME}/{APP_NAME}/serve/

In [ ]:
!gsutil ls -R gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/pipeline_root/rthallam/560224572293/pytorch-text-classifier-pipeline-20210921044652/submit-custom-training-job_-3608043008500432896/model/model/finetuned-bert-classifier/

In [11]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build"],
    output_component_file="./pipelines/build_custom_serving_image.yaml",
)
def build_custom_serving_image(
    project: str,
    gs_model_artifacts: Input[Model],
    gs_serving_dependencies_path: str,
    model_display_name: str
)-> NamedTuple(
    "Outputs",
    [
        ("serving_container_uri", str)
    ],
):

    from datetime import datetime
    from collections import namedtuple
    import logging
    import os

    from google.cloud.devtools import cloudbuild_v1 as cloudbuild

    logging.getLogger().setLevel(logging.INFO)
    build_client = cloudbuild.services.cloud_build.CloudBuildClient()

    logging.info(f"gs_serving_dependencies_path: {gs_serving_dependencies_path}")
    logging.info(f"gs_model_artifacts.uri: {gs_model_artifacts.uri}")
    logging.info(f"gs_model_artifacts.path: {gs_model_artifacts.path}")

    gs_dockerfile_path = os.path.join(gs_serving_dependencies_path, 'Dockerfile')
    gs_predictor_src_path = os.path.join(gs_serving_dependencies_path, 'predictor/')

    build_version = datetime.now().strftime("%Y%m%d%H%M%S")
    image_name = f"pytorch_predict_{model_display_name}"
    serving_image_uri = f"gcr.io/{project}/{image_name}:{build_version}"
    logging.info(f"serving_image_uri: {serving_image_uri}")

    build = cloudbuild.Build(
        images=[serving_image_uri]
    )
    build.steps = [
        {
            "name": "gcr.io/cloud-builders/gcloud",
            "entrypoint": "bash",
            "args": ["-c", "mkdir model"]
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", "-r", f"{gs_model_artifacts.uri}/model/{model_display_name}/", "model/"]
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", f"{gs_predictor_src_path}custom_text_handler.py", f"{gs_predictor_src_path}index_to_name.json", "."]
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", gs_dockerfile_path, "Dockerfile"]
        },
        {
            "name": "gcr.io/cloud-builders/gcloud",
            "entrypoint": "bash",
            "args": ["-c", "ls -ltr"]
        },
        {
          "name": 'gcr.io/cloud-builders/docker',
          "args": [ 'build', '-t', serving_image_uri, '.' ]
        }
    ]
    operation = build_client.create_build(project_id=project, build=build, timeout=1800)
    logging.info("IN PROGRESS:")
    logging.info(operation.metadata)

    result = operation.result()
    # Print the completed status
    logging.info("RESULT:", result.status)

    outputs = namedtuple("Outputs", ["serving_container_uri"])

    return outputs(serving_image_uri)

### 4. Component to test model deployment making online prediction requests

In [12]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build", "google-cloud-aiplatform"],
    output_component_file="./pipelines/make_prediction_request.yaml")
def make_prediction_request( 
    project: str,
    bucket: str,
    endpoint: Input[Artifact],
    instances: List
    ):
    from datetime import datetime
    import logging
    import base64

    from google.cloud import aiplatform

    logging.getLogger().setLevel(logging.INFO)
    aiplatform.init(project=project, staging_bucket=bucket)
    logging.info(f"Endpoint URI: {endpoint.uri}")
    _endpoint = aiplatform.Endpoint(endpoint.uri.split("/")[-1])

    for instance in instances:
        if not isinstance(instance, (bytes, bytearray)):
            instance = instance.encode()
        logging.info(f"Input text: {instance.decode('utf-8')}")
        b64_encoded = base64.b64encode(instance)
        test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
        response = _endpoint.predict(instances=test_instance)
        logging.info(f"Prediction response: {response.predictions}")

In [77]:
@component(
    base_image="python:3.7",
    packages_to_install=["google-cloud-aiplatform", "pandas"],
    output_component_file="./pipelines/get_pipeline_metric_runs.yaml")
def get_pipeline_metric_runs(
    project: str,
    bucket: str,
    pipeline_name: str,
    pipeline_job_id: str
) -> NamedTuple(
    "Outputs",
    [
        ("metrics", dict)
    ],
):
    from google.cloud import aiplatform
    from collections import namedtuple
    import time

    # initialize vertex sdk
    aiplatform.init(
        project=project, 
        staging_bucket=bucket
    )

    time.sleep(60)

    # Get all metrics from all pipeline runs
    df_metrics = aiplatform.get_pipeline_df(pipeline_name)
    print(f"df_metrics.shape {df_metrics.shape}")

    columns = ['pipeline_name', 'run_name'] + \
        [col for col in df_metrics if col.startswith('metric.')]
    metrics = df_metrics[(df_metrics.pipeline_name==pipeline_name) & (df_metrics.run_name==pipeline_job_id)][columns]
    print(metrics)
    print(f"metrics.shape {metrics.shape}")
    metrics_dict = metrics.to_dict(orient='index')
    print(f"metrics_for_all_pipelines: {metrics_dict}")

    outputs = namedtuple("Outputs", ["metrics"])

    return outputs(metrics_dict)

## Define Pipeline

In [78]:
@dsl.pipeline(
    name="pytorch-text-classifier-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pytorch_text_classifier_pipeline(
    project: str,
    bucket: str,
    region: str,
    gs_train_script_path: str,
    gs_serving_dependencies_path: str,
    model_display_name: str,
    serving_container_health_route: str,
    serving_container_predict_route: str,
    serving_container_ports: str,
    eval_acc_threshold: float,
    pipeline_job_id: str,
    pipeline_name: str,
    is_hp_tuning_enabled: str = 'n',
):
    # build custom container for training job passing the 
    # GCS location of the training application code 
    build_custom_train_image_task = build_custom_train_image(
        project=project,
        gs_train_src_path=gs_train_script_path,
        model_display_name=model_display_name
    ).set_caching_options(True).set_display_name("Build custom training image")

    # train the model on Vertex AI by submitting a CustomJob 
    # using the custom container
    training_container_image_uri = build_custom_train_image_task.outputs["training_container_uri"]
    run_train_task = submit_custom_training_job(
        project=project,
        bucket=bucket,
        training_container_image_uri=training_container_image_uri,
        eval_metric_key="eval_accuracy",
        model_display_name=model_display_name
    ).set_caching_options(True).set_display_name("Run custom training job")

    with dsl.Condition((is_hp_tuning_enabled == "n" and run_train_task.outputs["eval_metric"] > eval_acc_threshold), name="Model deploy decision"):
        # build custom container for serving predictions using 
        # the trained model artifacts served by TorchServe
        build_custom_serving_image_task = build_custom_serving_image(
            project=project,
            gs_model_artifacts=run_train_task.outputs["model"],
            gs_serving_dependencies_path=gs_serving_dependencies_path,
            model_display_name=model_display_name
        ).set_caching_options(True).set_display_name("Build custom serving image")

        # upload model to vertex ai
        # NOTE: model artifacts and the prediction handler are part of the container
        serving_container_image_uri = build_custom_serving_image_task.outputs['serving_container_uri']
        model_upload_task = aip_components.ModelUploadOp(
            project=project, 
            display_name=model_display_name,
            serving_container_image_uri=serving_container_image_uri,
            serving_container_predict_route=serving_container_predict_route,
            serving_container_health_route=serving_container_health_route,
            serving_container_ports=serving_container_ports
        ).set_caching_options(True).set_display_name("Upload model")

        # create endpoint to deploy one or more models
        # An endpoint provides a service URL where the prediction requests are sent
        endpoint_create_task = aip_components.EndpointCreateOp(
            project=project,
            display_name=model_display_name,
        ).set_caching_options(True).set_display_name("Create endpoint")

        # deploy models to endpoint to associates physical resources with the model 
        # so it can serve online predictions 
        model_deploy_task = aip_components.ModelDeployOp(
            project=project,
            endpoint=endpoint_create_task.outputs["endpoint"],
            model=model_upload_task.outputs["model"],
            deployed_model_display_name=model_display_name,
            machine_type="n1-standard-4",
        ).set_caching_options(True).set_display_name("Deploy model to endpoint")

        # test model deployment by making online prediction requests
        test_instances = [
            "Jaw dropping visual affects and action! One of the best I have seen to date.",
            "Take away the CGI and the A-list cast and you end up with film with less punch.",
        ]
        predict_test_instances_task = make_prediction_request(
            project,
            bucket,
            model_deploy_task.outputs["endpoint"],
            test_instances
        ).set_caching_options(True).set_display_name("Test model deployment making online predictions")

        current_run_metrics = get_pipeline_metric_runs(
            project,
            bucket,
            pipeline_name,
            pipeline_job_id
        ).set_caching_options(True).set_display_name("Show current run metrics")
        current_run_metrics.after(run_train_task)

In [79]:
PIPELINE_JSON_SPEC_PATH = './pipelines/pytorch_text_classifier_pipeline_spec.json'
compiler.Compiler().compile(
    pipeline_func=pytorch_text_classifier_pipeline,
    package_path=PIPELINE_JSON_SPEC_PATH)

In [80]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [81]:
PIPELINE_NAME = f"pytorch-text-classifier-pipeline"
PIPELINE_JOB_ID = f"pipeline-{APP_NAME}-{get_timestamp()}"
TRAIN_APP_CODE_PATH = f"{BUCKET_NAME}/{APP_NAME}/train/"
SERVE_DEPENDENCIES_PATH = f"{BUCKET_NAME}/{APP_NAME}/serve/"
pipeline_params = {
    "gs_train_script_path": TRAIN_APP_CODE_PATH,
    "gs_serving_dependencies_path": SERVE_DEPENDENCIES_PATH,
    "project": PROJECT_ID,
    "bucket": BUCKET_NAME,
    "region": REGION,
    "model_display_name": APP_NAME,
    "serving_container_health_route": "/ping",
    "serving_container_predict_route": f"/predictions/{APP_NAME}",
    "serving_container_ports": [7080],
    "eval_acc_threshold": 0.88,
    "is_hp_tuning_enabled": "n",
    "pipeline_name": PIPELINE_NAME,
    "pipeline_job_id": PIPELINE_JOB_ID
}

In [82]:
pipeline_job = pipeline_jobs.PipelineJob(
    display_name=PIPELINE_NAME,
    job_id=PIPELINE_JOB_ID,
    template_path=PIPELINE_JSON_SPEC_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=pipeline_params,
    enable_caching=True
)

In [83]:
response = pipeline_job.run(sync=False)
response

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/560224572293/locations/us-central1/pipelineJobs/pipeline-finetuned-bert-classifier-20210923041546
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/560224572293/locations/us-central1/pipelineJobs/pipeline-finetuned-bert-classifier-20210923041546')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipeline-finetuned-bert-classifier-20210923041546?project=560224572293


In [73]:
import time
time.sleep(2)

In [ ]:
df = aiplatform.get_pipeline_df(pipeline=PIPELINE_NAME.replace("_", "-"))
df

In [21]:
metric_columns = [col for col in df if col.startswith('metric.')]
pipeline_columns = ['pipeline_name', 'run_name']
cols = pipeline_columns + metric_columns
cols

['pipeline_name',
 'run_name',
 'metric.epoch',
 'metric.eval_samples_per_second',
 'metric.eval_accuracy',
 'metric.eval_loss',
 'metric.eval_steps_per_second',
 'metric.eval_runtime']

In [92]:
while True:
    df = aiplatform.get_pipeline_df(pipeline=PIPELINE_NAME.replace("_", "-"))
    pipeline_name = PIPELINE_NAME.replace("_", "-")
    pipeline_job_id = PIPELINE_JOB_ID
    metrics = df[(df.pipeline_name==pipeline_name) & (df.run_name==pipeline_job_id)][cols]
    metrics_dict = metrics.to_dict(orient='index')
    print(metrics_dict)

{0: {'pipeline_name': 'pytorch-text-classifier-pipeline', 'run_name': 'pipeline-finetuned-bert-classifier-20210923041546', 'metric.epoch': nan, 'metric.eval_samples_per_second': nan, 'metric.eval_accuracy': nan, 'metric.eval_loss': nan, 'metric.eval_steps_per_second': nan, 'metric.eval_runtime': nan}}
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/560224572293/locations/us-central1/pipelineJobs/pipeline-finetuned-bert-classifier-20210923041546 current state:
PipelineState.PIPELINE_STATE_RUNNING
{0: {'pipeline_name': 'pytorch-text-classifier-pipeline', 'run_name': 'pipeline-finetuned-bert-classifier-20210923041546', 'metric.epoch': nan, 'metric.eval_samples_per_second': nan, 'metric.eval_accuracy': nan, 'metric.eval_loss': nan, 'metric.eval_steps_per_second': nan, 'metric.eval_runtime': nan}}
{0: {'pipeline_name': 'pytorch-text-classifier-pipeline', 'run_name': 'pipeline-finetuned-bert-classifier-20210923041546', 'metric.epoch': nan, 'metric.eval_samples_per_second': na

KeyboardInterrupt: 